In [2]:
#%%capture
#!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.5.0/spark-sql-kafka-0-10_2.12-3.5.0.jar"
#!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-10_2.12/3.5.0/spark-streaming-kafka-0-10_2.12-3.5.0.jar"

In [3]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0 pyspark-shell'

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [5]:
spark = SparkSession.builder.appName("WindowApp").getOrCreate()

lines = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "10.224.177.98:8097") \
  .option("subscribe", "input") \
  .option("startingOffsets", "latest") \
  .load()

In [6]:
lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [7]:
#Lab 2.1
schema = StructType() \
    .add("event_time", TimestampType()) \
    .add("message", StringType())

parsed = lines.selectExpr("CAST(value AS STRING) as json_str") \
    .select(from_json(col("json_str"), schema).alias("data")) \
    .select("data.event_time", "data.message")

parsed.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- message: string (nullable = true)



In [8]:
words = parsed.select(
   explode(
       split(parsed.message, " ")
   ).alias("word"),
   col("event_time")
)

In [9]:
words.printSchema()

root
 |-- word: string (nullable = false)
 |-- event_time: timestamp (nullable = true)



In [27]:
# Tumbling windowing
if False:
    wordCounts = words.groupBy(
        window("event_Time", "5 seconds"), # win length, slide length
        "word"
    ).count()

In [39]:
# Sliding windowing 
if True:
    wordCounts = words.groupBy(
        window("event_Time", "10 minutes", "5 minutes"),
        "word"
    ).count()

In [40]:
wordCounts.printSchema()

root
 |-- window: struct (nullable = true)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- word: string (nullable = false)
 |-- count: long (nullable = false)



In [41]:
if True:
    query = wordCounts \
      .writeStream \
      .trigger(processingTime='1 seconds') \
      .outputMode("complete") \
      .format("console") \
      .option("truncate", False) \
      .start()
    
    query.awaitTermination()
#Try to use update

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [38]:
for q in spark.streams.active:
    q.stop()